## Download the dataset
In this example, we will use the Imagenette dataset to demonstrate how to prepare data for active learning.

Before we start, we need to prepare 3 sets of data:

- Initial samples: A dataset of labeled images to train an initial model. If you don't have any labeled data, you can label some images yourself.
- Unlabeled samples: A dataset of unlabeled images. We will continuously sample from this set using active learning strategies.
- Evaluation samples: A dataset of labeled images. We will use this set to evaluate the performance of the model. This is the test set, DO NOT use it for active learning. Split this out in the beginning.

We will use the Imagenette dataset as a working example in this notebook.

First, lets download the dataset and extract it.

In [1]:
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
!tar -xvzf imagenette2.tgz
!mv imagenette2 data/imagenette
!rm imagenette2.tgz

## Load the dataset

`active-vision` currently supports datasets in a pandas dataframe format. The dataframe should have at least 2 columns: `filepath` and `label`.

In [1]:
from fastai.vision.all import get_image_files

path = "data/imagenette/train"
image_files = get_image_files(path)
len(image_files)

9469

In [2]:
lbl_dict = {
    "n01440764": "tench",
    "n02102040": "English springer",
    "n02979186": "cassette player",
    "n03000684": "chain saw",
    "n03028079": "church",
    "n03394916": "French horn",
    "n03417042": "garbage truck",
    "n03425413": "gas pump",
    "n03445777": "golf ball",
    "n03888257": "parachute",
}


In [3]:
import pandas as pd

# Create a dataframe with the filepath and label from parent directory
labels = [str(path.parts[-2]) for path in image_files]

# Map the labels to the label dictionary
labels = [lbl_dict[lbl] for lbl in labels]

df = pd.DataFrame({"filepath": [str(path) for path in image_files], "label": labels})

df

,filepath,label
0,data/imagenette/train/n03394916/ILSVRC2012_val_00046669.JPEG,French horn
1,data/imagenette/train/n03394916/n03394916_58454.JPEG,French horn
2,data/imagenette/train/n03394916/n03394916_32588.JPEG,French horn
3,data/imagenette/train/n03394916/n03394916_33663.JPEG,French horn
4,data/imagenette/train/n03394916/n03394916_27948.JPEG,French horn
...,...,...
9464,data/imagenette/train/n02979186/n02979186_8089.JPEG,cassette player
9465,data/imagenette/train/n02979186/n02979186_19444.JPEG,cassette player
9466,data/imagenette/train/n02979186/n02979186_11074.JPEG,cassette player
9467,data/imagenette/train/n02979186/n02979186_2938.JPEG,cassette player


## Initial samples
As an initial step, we will randomly sample 10 samples from each class. We will use these samples to kickstart the active learning process.

In [4]:
initial_samples = (
    df.groupby("label")
    .apply(lambda x: x.sample(n=10, random_state=316))
    .reset_index(drop=True)
)

initial_samples

/var/folders/9y/5mpk58851fq38f8ljx2svvnm0000gn/T/ipykernel_22109/414664958.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=10, random_state=316))


,filepath,label
0,data/imagenette/train/n02102040/n02102040_2788.JPEG,English springer
1,data/imagenette/train/n02102040/n02102040_3759.JPEG,English springer
2,data/imagenette/train/n02102040/n02102040_1916.JPEG,English springer
3,data/imagenette/train/n02102040/n02102040_6147.JPEG,English springer
4,data/imagenette/train/n02102040/n02102040_403.JPEG,English springer
...,...,...
95,data/imagenette/train/n01440764/n01440764_10043.JPEG,tench
96,data/imagenette/train/n01440764/n01440764_31535.JPEG,tench
97,data/imagenette/train/n01440764/n01440764_12848.JPEG,tench
98,data/imagenette/train/n01440764/n01440764_3997.JPEG,tench


Let's check the distribution of the labels.

In [5]:
initial_samples["label"].value_counts()

label
English springer    10
French horn         10
cassette player     10
chain saw           10
church              10
garbage truck       10
gas pump            10
golf ball           10
parachute           10
tench               10
Name: count, dtype: int64

And save it to a parquet file.

In [6]:
initial_samples.to_parquet("initial_samples.parquet")

## Unlabeled samples
For the remaining samples, we will use them as unlabeled samples. We will sample from these samples using active learning strategies.


In [7]:
# Get the remaining samples by using pd.Index.difference
remaining_samples = df[~df.index.isin(initial_samples.index)].reset_index(drop=True)
remaining_samples


,filepath,label
0,data/imagenette/train/n03394916/n03394916_4437.JPEG,French horn
1,data/imagenette/train/n03394916/n03394916_42413.JPEG,French horn
2,data/imagenette/train/n03394916/n03394916_38808.JPEG,French horn
3,data/imagenette/train/n03394916/n03394916_24128.JPEG,French horn
4,data/imagenette/train/n03394916/n03394916_11289.JPEG,French horn
...,...,...
9364,data/imagenette/train/n02979186/n02979186_8089.JPEG,cassette player
9365,data/imagenette/train/n02979186/n02979186_19444.JPEG,cassette player
9366,data/imagenette/train/n02979186/n02979186_11074.JPEG,cassette player
9367,data/imagenette/train/n02979186/n02979186_2938.JPEG,cassette player


In [8]:
remaining_samples.to_parquet("unlabeled_samples.parquet")

## Evaluation samples

Now let's create the evaluation samples which will be used to evaluate the performance of the model. We will use the validation set from the Imagenette dataset as the evaluation set.



In [9]:
path = "data/imagenette/val"
image_files = get_image_files(path)
len(image_files)


3925

In [10]:
labels = [str(path.parts[-2]) for path in image_files]

# Map the labels to the label dictionary
labels = [lbl_dict[lbl] for lbl in labels]

evaluation_samples = pd.DataFrame(
    {"filepath": [str(path) for path in image_files], "label": labels}
)

evaluation_samples

,filepath,label
0,data/imagenette/val/n03394916/n03394916_32422.JPEG,French horn
1,data/imagenette/val/n03394916/n03394916_69132.JPEG,French horn
2,data/imagenette/val/n03394916/n03394916_33771.JPEG,French horn
3,data/imagenette/val/n03394916/n03394916_29940.JPEG,French horn
4,data/imagenette/val/n03394916/ILSVRC2012_val_00033682.JPEG,French horn
...,...,...
3920,data/imagenette/val/n02979186/n02979186_27392.JPEG,cassette player
3921,data/imagenette/val/n02979186/n02979186_2742.JPEG,cassette player
3922,data/imagenette/val/n02979186/n02979186_2312.JPEG,cassette player
3923,data/imagenette/val/n02979186/n02979186_12822.JPEG,cassette player


In [11]:
evaluation_samples.to_parquet("evaluation_samples.parquet")

We are now ready to start the active learning process.